# Introduction to Pyomo for WaterTAP Users

### [WaterTAP](https://watertap.readthedocs.io/en/stable/) is built on [IDAES](https://idaes-pse.readthedocs.io/en/stable/), which itself is built on [Pyomo](https://pyomo.readthedocs.io/en/stable/). 

Pyomo is software package for formulating, solving, and analyzing optimization models. Knowing how to interact with these components is important for being able to use WaterTAP.

The fundamental Pyomo components used throughout WaterTAP are:
- `Var`: unknown values in the model, i.e., values we are trying to optimize in our model.
- `Constraint`: equations relating model variables and parameters to each other.
- `Param`: data that must be provided in order to find optimal values for the decision variables.
- `Expression`: non-constraint relationship between modeling components.

This tutorial goes step-by-step of creating a few different Pyomo models and highlights some important syntax and methods to be aware of when interacting with these Pyomo components in WaterTAP models.

> ### Note: This tutorial intends to introduce some important concepts and components from Pyomo that are used extensively in WaterTAP. Implementation of models in WaterTAP is slightly different.

# Make necessary imports

The imports below represent the most commonly used Pyomo components in WaterTAP.

In [ ]:
from pyomo.environ import (
    ConcreteModel,
    Var,
    Set,
    Param,
    Constraint,
    Expression,
    check_optimal_termination,
    assert_optimal_termination,
    value,
    NonNegativeReals,
    SolverFactory,
    exp,
    units as pyunits,
)
from pyomo.util.check_units import assert_units_consistent

from idaes.core import FlowsheetBlock
from idaes.core.util.model_statistics import degrees_of_freedom

from watertap.core.solvers import get_solver

# Creating Pyomo model

The `ConcreteModel` is the base object for a Pyomo and WaterTAP model. All other variables, parameters, constraints, etc. are built upon this object.
    
No arguments are necessary. Convention is to call the local object `m` or `model`, but it can be called whatever you want.


In [ ]:
# Instantiate model as instance of a ConcreteModel object
m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)  # Create a flowsheet block

# Displaying the model

We call the `.display()` method to see variables, objectives, and constraints on the model.

In [ ]:
# Call the display method to show variables, objectives, and constraints on the model
m.fs.display()

# Adding variables

To represent the different legs of the right triangle, we add three variables - `A`, `B`, and `C` as `Var`

Common (and optional) keyword arguments to `Var`:
- `initialize` - the initial value for the variable
- `domain` - possible values the variable can take
- `bounds` - limits on the values the variable can take
- `units` - physical dimensions for the variable (note: more detail on units is provided later in this tutorial)
- `doc` - description of the variable

In [ ]:
m.fs.A = Var(
    initialize=10,
    domain=NonNegativeReals,
    bounds=(0, None),
    units=pyunits.m,
    doc="Model variable A for one leg of a right triangle",
)
m.fs.B = Var(
    initialize=10,
    domain=NonNegativeReals,
    bounds=(0, None),
    units=pyunits.m,
    doc="Model variable B for the other leg of a right triangle",
)
m.fs.C = Var(
    initialize=10,
    domain=NonNegativeReals,
    bounds=(0, None),
    units=pyunits.m,
    doc="Model variable C for the hypotenuse of a right triangle",
)

m.fs.display()

# Adding constraints

Now we add constraints relating our three variables with the Pythagorean Theorem:

### $A^2 + B^2 = C^2$

The relationship for a `Constraint` can be added in a few ways. Here, we use the `expr` keyword to directly implement the relationship. 

Note that we are using the *equality operator* (`==`) rather than the *assignment operator* (`=`).

In [ ]:
m.fs.pythagorean = Constraint(
    expr=m.fs.A**2 + m.fs.B**2 == m.fs.C**2, doc="Pythagorean theorem"
)

# Check degrees of freedom (DOF)

The degrees of freedom for the model are determined using the `degrees_of_freedom()` function.

In [ ]:
dof = degrees_of_freedom(m)
print(f"Degrees of Freedom: {dof}")

With two DOF, we can add another `Constraint` to further constrain the problem.

Let's say we know `A` and `B` are related with:

### $ A = P \times B $

We add a `Param` to represent the ratio between `A` and `B`. The `Param` has some of the same keywords as `Var`. Here we specify `mutable=True` to indicate that we want to be able to change the parameter value after construction.

In [ ]:
m.fs.P = Param(
    initialize=3,
    mutable=True,
    units=pyunits.dimensionless,
    doc="Ratio between two legs of traiangle",
)


# An alternative way to define the constraint is via a decorator.
# This is equivalent to the approach we took earlier to define m.pythagorean.
@m.fs.Constraint(doc="Equation relating A and B")
def A_B_relation(b):
    return b.A == b.B * b.P


# m.A_B_relation = Constraint(expr=m.A == m.B * m.P, doc="Equation relating A and B")

# You can use both the display and pprint methods on Vars, Params, and Constraints as well.
m.fs.A_B_relation.pprint()
m.fs.A_B_relation.display()

dof = degrees_of_freedom(m)
print(f"Degrees of Freedom: {dof}")

# Fixing variables

At this point, the model still has 1 DOF. We can specify the length of one of our triangle sides using the `.fix()` method on a `Var`

In [ ]:
m.fs.A.fix(5)

dof = degrees_of_freedom(m)
print(f"Degrees of Freedom: {dof}")

# We can just as easily unfix the variable:
m.fs.A.unfix()

dof = degrees_of_freedom(m)
print(f"Degrees of Freedom: {dof}")

# Solving the model

Now we create the solver by calling `get_solver()`. Alternatively, we could use the version of IPOPT from Pyomo's `SolverFactory`.

In [ ]:
# solver = SolverFactory("ipopt")
solver = get_solver()

# Fix A again
m.fs.A.fix(5)

dof = degrees_of_freedom(m)
assert dof == 0

results = solver.solve(m)

# Checking solver results

The solve didn't give us an error, but we don't know if the solve is optimal.
Information about the solver termination is stored in the local variable `results`. We can check optimality with:
- the `assert_optimal_termination` function
- the `check_optimal_termination` function
- printing the `termination_condition` stored in `results`
- simply printing `results`

In [ ]:
# Nothing to print, but will raise an error if not optimal
assert_optimal_termination(results)

# Returns a boolean
is_optimal = check_optimal_termination(results)
print(f"Is optimal: {is_optimal}")

# Print termination condition
termination_condition = results.solver.termination_condition
print(f"Termination condition: {termination_condition}")

# Print the full results object
print(results)

### For `Param` components, we change the value using the `set_value()` method

(The `.fix()` method will yield an error.)

In [ ]:
m.fs.P.set_value(23)
dof = degrees_of_freedom(m)
print(f"Degrees of Freedom: {dof}")

results = solver.solve(m)
assert_optimal_termination(results)

m.fs.display()

# Accessing model information

You can access the value of a `Var` or `Param` by using the `value()` function.

In [ ]:
print(f"A = {value(m.fs.A)}", f"\nB = {value(m.fs.B)}", f"\nC = {value(m.fs.C)}")

# Indexing model components

Many Pyomo components can be indexed with any number of indexes. For example, if your model is dynamic (non-steady state) and you are tracking multiple constituents, you could index your `Var`s and `Constraint`s to both time and your constituents.

When constructing your model components, the first non-keyword argument(s) are your indexes. You can pass any iterable as an index. In the example below, we create an initial concentration variable `C_0` that is indexed to a set of `ions`. Here, we use the Pyomo `Set` component, but we could use e.g. a Python list or numpy array just the same.


In [ ]:
m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)  # Create a flowsheet block

# Create index
ions = ["Na", "Mg", "Ca", "Sr"]

m.fs.ions = Set(initialize=ions, doc="Set of ions")

m.fs.C_0 = Var(
    m.fs.ions,
    initialize=2,
    bounds=(0, None),
    units=pyunits.mg / pyunits.L,
    doc="Initial ion concentration",
)

To initialize an indexed `Var`, you can pass a single value or any Python iterable (e.g. dictionary) that maps the indexes to their initial value. 

In the example above, we say `initialize=2`, so the initial concentration for all my ions will be 2 mg/L. If we want different initial values for each index, we could use a dictionary.


In [ ]:
m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)  # Create a flowsheet block

# Create index and initialization values.
ions = ["Na", "Mg", "Ca", "Sr"]
ions_init = {"Na": 1.4, "Mg": 2.2, "Ca": 3.0, "Sr": 4.2}
m.fs.ions = Set(initialize=ions, doc="Set of ions")

m.fs.C_0 = Var(
    m.fs.ions,
    initialize=ions_init,
    bounds=(0, None),
    units=pyunits.mg / pyunits.L,
    doc="Initial ion concentration",
)
m.fs.C_0.display()

Similarly, a `Constraint` is indexed by passing the index as the first non-keyword argument(s).

Let's say we can get the steady-state concentration of our ions by multiplying it by some constant parameter `K_eq` that is different for each ion. We would create a steady-state concentration `Var` (`C_ss`) and define our equilibrium calculation `Constraint` that is indexed to each ion:

In this model, we also include an `Expression` to report the change in concentration between the intial and steady state concentrations (`deltaC`).

In [ ]:
K_eq_init = {"Na": 12.12, "Mg": 5.555, "Ca": 3.2, "Sr": 0.349}

m.fs.K_eq = Var(ions, initialize=K_eq_init, doc="Equilibrium constant")

m.fs.C_ss = Var(
    m.fs.ions,
    initialize=ions_init,
    bounds=(0, None),
    units=pyunits.mg / pyunits.L,
    doc="Steady-state ion concentration",
)

# There are several ways to construct constraints with Pyomo.
# A few are presented here for illustrative purposes.
# In all cases, the index is the first argument passed to the Constraint constructor.

# 1. Using a function and the `rule=` keyword
# def eq_steady_state_conc(m, i):
#     # The first argument must be the model (m), and the second must be the index
#     return m.C_ss[i] == m.C_0[i] * exp(-m.K_eq[i])
# m.steady_state_conc_constr = Constraint(ions, rule=eq_steady_state_conc)

# 2. Using a lambda function and the `expr=` or `rule=` keyword
m.fs.steady_state_conc_constr = Constraint(
    m.fs.ions,
    # rule=lambda m, i: m.C_ss[i] == m.C_0[i] * exp(-m.K_eq[i]),
    expr=lambda b, i: b.C_ss[i] == b.C_0[i] * exp(-b.K_eq[i]),
)

m.fs.deltaC = Expression(
    m.fs.ions,
    expr=lambda b, i: b.C_0[i] - b.C_ss[i],
    doc="Change in concentration from initial to steady state",
)

m.fs.C_0.fix()
m.fs.K_eq.fix()
dof = degrees_of_freedom(m)
print(f"Degrees of Freedom: {dof}")

results = solver.solve(m)
assert_optimal_termination(results)
assert_units_consistent(m)

# m.display()

# You can iterate through the index set to access variable values
# To access the value of indexed components, you place the index in brackets [ ] after the component name
for ion in m.fs.ions:
    print(f"Ion {ion}:")
    print(
        f"  Initial Concentration (C_0): {value(m.fs.C_0[ion]):.2f} {pyunits.get_units(m.fs.C_0[ion])}"
    )
    print(
        f"  Equilibrium Constant (K_eq): {value(m.fs.K_eq[ion]):.2f} {pyunits.get_units(m.fs.K_eq[ion])}"
    )
    print(
        f"  Steady-State Concentration (C_ss): {value(m.fs.C_ss[ion]):.2e} {pyunits.get_units(m.fs.C_ss[ion])}"
    )
    print(
        f"  Change in Concentration (deltaC): {value(m.fs.deltaC[ion]):.2f} {pyunits.get_units(m.fs.deltaC[ion])}"
    )

## Using `pyunits`

Pyomo (and IDAES and WaterTAP) make use of a package called [Pint](https://pint.readthedocs.io/en/stable/), which we have imported as `pyunits`.

`pyunits` are extremely useful for making easy unit conversions but also ensuring our models have consistent units.

We can define any value with any dimensions:
Let's create 100 g/L of some substance

In [ ]:
C = 100 * pyunits.g / pyunits.L

print(C)
print(value(C))
print(C())

## Unit conversions

We can convert between units using `pyunits.convert()` as long as they are of the same kind.

And you can get units from any variable using `pyunits.get_units()`

In [ ]:
C2 = pyunits.convert(C, to_units=pyunits.lb / pyunits.ft**3)
C3 = pyunits.convert(C, to_units=pyunits.grain / (pyunits.acre * pyunits.foot))

print(C2(), pyunits.get_units(C2))
print(C3(), pyunits.get_units(C3))

### Units carry through operations

In [ ]:
Q = 42 * pyunits.m**3 / pyunits.hr
conc = 100 * pyunits.mg / pyunits.L

mass_flow = Q * conc

print(mass_flow(), pyunits.get_units(mass_flow))

# Or

mass_flow = pyunits.convert(Q * conc, to_units=pyunits.g / pyunits.s)

print(mass_flow(), pyunits.get_units(mass_flow))